DESCRIPTION: 
Cleaning.ipynb is focused on preparing the raw dataset for analysis. It imports the raw files, handles issues like missing values, duplicates, and inconsistent entries, and standardizes the structure of the data. The notebook likely also filters out irrelevant or erroneous records and organizes the dataset into a clean, consistent format that can be reused. By the end of this notebook, the data is ready for analysis and may be saved into a cleaned CSV or database table.

### Dropping Columns that have low quality data and are less important for our EDA
### Dropping Repeated Product_id Values

In [ ]:
import pandas as pd
import os


def drop_unnecessary_columns(input_file: str):
    """
    Reads a CSV file, drops a predefined list of columns, removes duplicate
    product_id entries, and returns the result.

    Args:
        input_file (str): The path to the input CSV file.
    """
    try:
        print(f"[*] Reading data from '{input_file}'...")
        # Use low_memory=False for large files to avoid data type warnings
        df = pd.read_csv(input_file, low_memory=False)
        print(f"[+] Successfully loaded {len(df)} rows.")

        # --- List of columns to be removed ---
        columns_to_drop = [
            'last_verified',
            'payment_terms',
            'delivery_time',  
            'minimum_order_quantity', 
            'supply_ability',
            'trust_stamp_url',
            'certifications',
            'made_in_india',
        ]
        
        # Check which of the columns to drop actually exist in the DataFrame
        existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
        
        if not existing_columns_to_drop:
            print("[INFO] None of the specified columns were found to drop.")
            df_clean = df.copy() # Work with a copy if no columns are dropped
        else:
            print(f"[*] Dropping the following columns: {existing_columns_to_drop}")
            # Drop the columns
            df_clean = df.drop(columns=existing_columns_to_drop)
            print(f"    Original number of columns: {len(df.columns)}")
            print(f"    New number of columns: {len(df_clean.columns)}")

        print("-" * 50)
        
        # --- Remove Duplicate product_id Entries ---
        print("[*] Removing duplicate entries based on 'product_id'...")
        rows_before_dedupe = len(df_clean)
        print(f"    Number of rows before deduplication: {rows_before_dedupe}")

        # Drop duplicates, keeping the first occurrence
        df_deduplicated = df_clean.drop_duplicates(subset=['product_id'], keep='first')
        
        rows_after_dedupe = len(df_deduplicated)
        print(f"    Number of rows after deduplication: {rows_after_dedupe}")
        
        duplicates_removed = rows_before_dedupe - rows_after_dedupe
        print(f"[+] Total duplicate product_id entries removed: {duplicates_removed}")
        print("-" * 50)

        print("data carried with df_cleaned_up")
        return df_deduplicated
    
    except FileNotFoundError:
        print(f"[ERROR] The file '{input_file}' was not found.")
    except KeyError:
        print("Error: The column 'product_id' was not found in the DataFrame.")
    except Exception as e:
        print(f"[ERROR] An unexpected error occurred: {e}")

# --- Main execution block ---
if __name__ == "__main__":
    # Define your input file here
    input_filename = r'C:\Users\yuvra\OneDrive\Desktop\Slooze-Assignment\Assignment\Data\Silver\data_raw.csv'
    
    
    # Run the function
    df_cleaned_up = drop_unnecessary_columns(input_filename)

[*] Reading data from 'Assignment\Data\Silver\data_raw.csv'...
[ERROR] The file 'Assignment\Data\Silver\data_raw.csv' was not found.


# corrected DataType

In [37]:
import pandas as pd
import os


def format_data_types():
    """
    Reads a data file, converts columns to optimal data types based on specific
    rules, and saves the result.
    """
    try:
        print(f"[*] Reading data from 'df_cleaned'")
        
        # --- Define columns for each data type category based on your rules ---
        
        # Rule 1: Identifiers should be strings
        string_id_cols = ['product_id', 'profile_id', 'userid']
        
        # Rule 2: Price columns should be floats, others should be nullable integers
        float_cols = ['min_price', 'max_price', 'fix_price']
        integer_cols = ['member_since', 'year_established', 'buyer_feedback_score']
        
        # Rule 3: Boolean columns
        boolean_cols = [
            'has_trust_stamp', 'is_distributor', 'is_supplier', 'is_exporter',
            'is_trader', 'is_service_provider', 'in_stock', 'is_manufacturer'
        ]
        
        # Rule 4: Categorical columns for memory efficiency
        category_cols = [
            'main_category', 'sub_category', 'city', 'state', 'country_name'
        ]

        print("[*] Converting data types for memory and performance optimization...")

        # --- Apply Conversions ---
        for col in string_id_cols:
            if col in df_cleaned_up.columns:
                df_cleaned_up[col] = df_cleaned_up[col].astype(str)

        for col in float_cols:
            if col in df_cleaned_up.columns:
                df_cleaned_up[col] = pd.to_numeric(df_cleaned_up[col], errors='coerce')

        for col in integer_cols:
            if col in df_cleaned_up.columns:
                # First convert to numeric, coercing errors to NaN
                numeric_series = pd.to_numeric(df_cleaned_up[col], errors='coerce')
                # Then convert to nullable integer type 'Int64'
                df_cleaned_up[col] = numeric_series.astype('Int64')

        for col in boolean_cols:
            if col in df_cleaned_up.columns:
                df_cleaned_up[col] = df_cleaned_up[col].astype('boolean')

        for col in category_cols:
            if col in df_cleaned_up.columns:
                df_cleaned_up[col] = df_cleaned_up[col].astype('category')
        
        # --- Final Report and Save ---
        print("\n--- Data Types After Conversion ---")
        df_cleaned_up.info(verbose=True, show_counts=True)
        print('data carried by df_corrected')
        return df_cleaned_up
        
        
       

    except FileNotFoundError:
        print(f"[ERROR] The file 'df_dropped' was not found.")
    except Exception as e:
        print(f"[ERROR] An unexpected error occurred: {e}")

# --- Main execution block ---
if __name__ == "__main__":
     
    # Run the main function
    df_corrected = format_data_types()


[*] Reading data from 'df_cleaned'
[*] Converting data types for memory and performance optimization...

--- Data Types After Conversion ---
<class 'pandas.core.frame.DataFrame'>
Index: 359964 entries, 0 to 367219
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   product_id            359964 non-null  object  
 1   profile_id            359964 non-null  object  
 2   userid                359964 non-null  object  
 3   product_name          359962 non-null  object  
 4   co_name               359959 non-null  object  
 5   prod_url              359964 non-null  object  
 6   profile_url           359964 non-null  object  
 7   main_category         359964 non-null  category
 8   sub_category          359964 non-null  category
 9   member_since          37481 non-null   Int64   
 10  year_established      269224 non-null  Int64   
 11  has_trust_stamp       359964 non-null  boolean 
 12  city  

### Price cleaning script Pandas


In [38]:
import pandas as pd
import re
import numpy as np
import os

# --- Configuration ---
USD_TO_INR_RATE = 87.27

# --- Helper functions (parse_price_string, process_price_columns, cleanup_price_columns) ---
# (These functions from your original script are assumed to be here and are correct)

def parse_price_string(price_input):
    """
    Parses a single string or number to extract price information.
    """
    if pd.isna(price_input) or not isinstance(price_input, str):
        if isinstance(price_input, (int, float)) and price_input > 0:
            return None, None, float(price_input)
        return None, None, None
    price_str = str(price_input).strip()
    multiplier = USD_TO_INR_RATE if '$' in price_str else 1.0
    found_patterns = re.findall(r'[\d,.]+', price_str)
    numbers = []
    for pattern in found_patterns:
        try:
            num = float(pattern.replace(',', ''))
            numbers.append(num)
        except ValueError:
            continue
    if not numbers:
        return None, None, None
    if '-' in price_str or 'to' in price_str.lower():
        if len(numbers) >= 2:
            min_p = min(numbers) * multiplier
            max_p = max(numbers) * multiplier
            return min_p, max_p, None
    if len(numbers) == 1:
        fix_p = numbers[0] * multiplier
        return None, None, fix_p
    return None, None, None

def process_price_columns(df):
    """
    Applies the full price extraction logic to the DataFrame.
    """
    def get_price_for_row(row):
        min_r = row.get('min_price_range', 0)
        max_r = row.get('max_price_range', 0)
        # Ensure min_r and max_r are numeric before comparison
        if pd.api.types.is_number(min_r) and pd.api.types.is_number(max_r):
            if min_r > 0 and max_r > 0 and max_r >= min_r:
                return min_r, max_r, np.nan
        
        priority_columns = ['price_string', 'price_numeric', 'price_from_sub_json']
        for col in priority_columns:
            price_val = row.get(col)
            if not pd.isna(price_val):
                min_p, max_p, fix_p = parse_price_string(price_val)
                if min_p is not None or fix_p is not None:
                    return min_p, max_p, fix_p
        return np.nan, np.nan, np.nan
    
    df[['min_price', 'max_price', 'fix_price']] = df.apply(get_price_for_row, axis=1, result_type='expand')
    return df

def cleanup_price_columns(df):
    """
    Removes the original, raw price columns from the DataFrame.
    """
    columns_to_drop = [
        'price_string', 'price_numeric', 'price_range',
        'min_price_range', 'max_price_range', 'price_from_sub_json'
    ]
    df_cleaned_price = df.drop(columns=columns_to_drop, errors='ignore')
    print("[*] Removed original price columns for a cleaner final dataset.")
    return df_cleaned_price


# --- REVISED FUNCTION ---
def format_price_columns_for_analysis(df):
    """
    Ensures price columns are purely numeric for analysis.
    """
    # 1. Ensure price columns are purely numeric. Any non-numeric becomes NaN.
    df['min_price'] = pd.to_numeric(df['min_price'], errors='coerce')
    df['max_price'] = pd.to_numeric(df['max_price'], errors='coerce')
    df['fix_price'] = pd.to_numeric(df['fix_price'], errors='coerce')
    
    print("[*] Ensured price columns are numeric for analysis.")
    return df

# --- Main processing function that uses the REVISED formatting function ---
def run_price_pipeline(input_df):
    """
    Takes a DataFrame, runs the full price cleaning and formatting pipeline,
    and returns the final processed DataFrame.
    """
    if not isinstance(input_df, pd.DataFrame):
        print("[ERROR] Input must be a pandas DataFrame.")
        return None
        
    print("[*] Starting price processing pipeline...")
    # Make a copy to avoid changing the original DataFrame
    df_processed = process_price_columns(input_df.copy()) 
    df_cleaned_price = cleanup_price_columns(df_processed)
    
    # Use the new, analysis-friendly formatting function
    df_final = format_price_columns_for_analysis(df_cleaned_price)
    
    print("[+] Price processing complete.")
    return df_final

# --- Example Main execution block for your notebook ---
if __name__ == '__main__':
    
    df_price_corrected = run_price_pipeline(df_corrected)



[*] Starting price processing pipeline...
[*] Removed original price columns for a cleaner final dataset.
[*] Ensured price columns are numeric for analysis.
[+] Price processing complete.


## Tried Cleaning yr_established (not very effective, % did not change)

In [39]:
import pandas as pd
import numpy as np
import os

def fill_year_established(input_df):
    """
    Takes a DataFrame, fills missing 'year_established' values based on 
    the 'member_since' column, and returns the modified DataFrame.
    """
    # Work on a copy to ensure the original DataFrame is not changed
    df = input_df.copy()

    # --- 1. Data Preparation ---
    # Ensure both columns are numeric, converting non-numbers to NaN (missing).
    df['member_since'] = pd.to_numeric(df['member_since'], errors='coerce')
    df['year_established'] = pd.to_numeric(df['year_established'], errors='coerce')

    # --- 2. Imputation Logic ---
    # Calculate the estimated year of establishment where possible.
    estimated_year = 2025 - df['member_since']

    # Use .fillna() to fill ONLY the missing 'year_established' values.
    # This now assigns the result back to the column, avoiding the warning.
    df['year_established'] = df['year_established'].fillna(estimated_year)

    # --- 3. Final Data Type Conversion ---
    # Convert the column to the nullable integer type 'Int64'.
    # Any remaining missing values will be correctly stored as pd.NA.
    df['year_established'] = df['year_established'].astype('Int64')
    
    print("[*] Filled missing 'year_established' values and set correct data type.")
    
    # Return the fully processed DataFrame
    return df

# --- Main execution block for your notebook ---
# This demonstrates how to use the corrected function.



# 2. Call the main function with your DataFrame
df_yr = fill_year_established(df_price_corrected)

[*] Filled missing 'year_established' values and set correct data type.


### Cleaning states and country names

In [44]:
import pandas as pd

# --- 1. Define the Comprehensive Cleaning Rules ---
# This dictionary is built directly from the list of unique values you provided.
state_mapping = {
    # Handle case variations
    'karnataka': 'Karnataka',
    
    # Handle old vs. new names
    'Pondicherry': 'Puducherry',
    
    # Handle merged territories
    'Dadra and Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman and Diu': 'Dadra and Nagar Haveli and Daman and Diu'
}

# --- 2. Apply the Cleaning ---
# Filter for India to get the 'before' count
india_df = df_price_corrected[df_price_corrected['country_name'] == 'India']
before_count = india_df['state'].nunique()
print(f"Number of unique states before cleaning: {before_count}")

# Apply the mapping to the entire 'state' column in the main DataFrame.
# We convert the column to a string type first to avoid the FutureWarning.
df_price_corrected['state_cleaned'] = df_price_corrected['state'].astype(str).replace(state_mapping)

# --- 3. Verify the Result ---
# Filter for India again, but use the new 'state_cleaned' column
india_df_cleaned = df_price_corrected[df_price_corrected['country_name'] == 'India']
after_count = india_df_cleaned['state_cleaned'].nunique()
print(f"Number of unique states after cleaning: {after_count}")




Number of unique states before cleaning: 38
Number of unique states after cleaning: 35


### tells % of data


In [47]:
import pandas as pd
import numpy as np

def calculate_completeness(df):
    """
    Calculates and prints the percentage of available (non-null) data 
    for each column in a pandas DataFrame after handling custom missing values.

    Args:
        df (pd.DataFrame): The input DataFrame to analyze.
    """
    # --- Input Validation ---
    if not isinstance(df, pd.DataFrame):
        print("[ERROR] Input must be a pandas DataFrame.")
        return

    if df.empty:
        print("[INFO] The DataFrame is empty. No analysis to perform.")
        return

    print(f"[*] Analyzing DataFrame with {len(df)} rows...")

    try:
        # --- Data Pre-processing ---
        # Create a copy to avoid modifying the original DataFrame sent to the function.
        df_cleaned = df.copy()

        # Define a list of values that should be treated as missing.
        values_to_replace = ['Not available','Not available','Negotiable', '-', 'unavailable', 'N/A', 0, '0']
        
        # Replace all specified values with NumPy's Not a Number (NaN).
        df_cleaned.replace(values_to_replace, np.nan, inplace=True)
        
        print("[*] Custom missing values ('Not available', '-', '0', etc.) have been handled.")

        # --- Core Calculation Logic ---
        # 1. Get the total number of rows (the denominator for our percentage).
        total_rows = len(df_cleaned)

        # 2. Count non-null values in each column of the cleaned data.
        non_null_counts = df_cleaned.count()

        # 3. Calculate the percentage of data available.
        #    This is the heart of the analysis.
        completeness_report = (non_null_counts / total_rows) * 100

        # 4. Sort the results for a more insightful report (most complete first).
        sorted_report = completeness_report.sort_values(ascending=False)

        # --- Display the Final Report ---
        print("\n" + "="*60)
        print("--- Column Data Completeness Report ---")
        print(f"{'Column Name':<40} | {'% Available Data'}")
        print("-" * 60)
        
        # Loop through the sorted series to print each column's completeness
        for column_name, percentage in sorted_report.items():
            print(f"{column_name:<40} | {percentage:>15.2f}%")
            
        print("="*60)

    except Exception as e:
        print(f"[ERROR] An unexpected error occurred during analysis: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    
    # Call the function to analyze your DataFrame
    calculate_completeness(df_yr)


[*] Analyzing DataFrame with 359964 rows...
[*] Custom missing values ('Not available', '-', '0', etc.) have been handled.

--- Column Data Completeness Report ---
Column Name                              | % Available Data
------------------------------------------------------------
product_id                               |          100.00%
profile_id                               |          100.00%
userid                                   |          100.00%
prod_url                                 |          100.00%
has_trust_stamp                          |          100.00%
profile_url                              |          100.00%
main_category                            |          100.00%
sub_category                             |          100.00%
is_distributor                           |          100.00%
is_supplier                              |          100.00%
is_exporter                              |          100.00%
is_manufacturer                          |          100

# saving to Parquet, Because parquet store column Dtypes and they are faster.


In [48]:
df_yr.to_csv(r'C:\Users\yuvra\OneDrive\Desktop\Slooze-Assignment\Assignment\Data\Gold\Data_processed.csv', index=False)
df_yr.to_parquet(r'C:\Users\yuvra\OneDrive\Desktop\Slooze-Assignment\Assignment\Data\Gold\Data_processed.parquet', index=False)